### Imports and setup

In [11]:
import pandas as pd
import pyreadr
from sklearn.calibration import CalibratedClassifierCV
# from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

pd.set_option("display.max_columns", None)

In [12]:
PATH = "lalonde.RData"
seed = 1234
num_trees = None

Y = "re78"
treat = "treat"
covar = [
    "age",
    "education",
    "black",
    "hispanic",
    "married",
    "nodegree",
    "re74",
    "re75",
    "u74",
    "u75",
]

raw = pyreadr.read_r(PATH)
# raw.keys()

## LDW-CPS1

### Data processing and propensity score

In [13]:
# Data preparation
# ================
ldw_co = raw["ldw_co"].copy()
ldw_co["treat"] = 1

ldw_cps_plus = pd.concat([raw["ldw_cps"], raw["ldw_co"]])

data = ldw_cps_plus.copy().reset_index(drop=True)
data.reset_index(inplace=True, names="row_idx")

# Estimating propensity score
# ==========================
X = data[covar]
y = data[treat]

model = RandomForestClassifier(
    n_estimators=50, max_depth=4, class_weight="balanced", random_state=seed
)
calib_model = CalibratedClassifierCV(model, cv=3, method="isotonic")
calib_model.fit(X, y)

data["ps"] = calib_model.predict_proba(X)[:, 1]

# model = LogisticRegression(C=1e6, max_iter=1000)
# model.fit(X, y)
# data['ps'] = model.predict_proba(X)[:, 1]

ldw_cps_trim = data[(data["ps"] < 0.9) & (data["sample"].isin([1, 3]))].copy()

In [14]:
data.shape, ldw_cps_trim.shape

((16437, 16), (16177, 16))

In [15]:
# Recalculate PS over trimmed data
# ================================

X = ldw_cps_trim[covar]
y = ldw_cps_trim[treat]

model = RandomForestClassifier(
    n_estimators=50, max_depth=4, class_weight="balanced", random_state=seed
)
calib_model = CalibratedClassifierCV(model, cv=3, method="isotonic")
calib_model.fit(X, y)

# model = LogisticRegression(C=1e6, max_iter=1000)
# model.fit(X, y)

ldw_cps_trim["propensity_score"] = calib_model.predict_proba(X)[:, 1]

___
**Facure p. 159:** There is also a bias-variance tradeoff when it comes to IPW. In general, the more precise the propensity score model, the lower the bias. However, a very precise model for $e(x)$ will generate a very imprecise effect estimate. This means you have to make your model precise enough to control for the bias, but not too much, or you will run into variance issues.
___

### Inverse probability weighting (IPW)

**Facure p. 159:** The more precise you make your model for $e(x)$ [...] you also make positivity less plausible [...] since you will concentrate the treatment in a los $e(x)$ region, far away from the controls and viceversa. 

In [16]:
weight_t = 1 / ldw_cps_trim.query("treat==1")["propensity_score"]
weight_nt = 1 / (1 - ldw_cps_trim.query("treat==0")["propensity_score"])

treated = ldw_cps_trim.query("treat==1")[Y]
ntreated = ldw_cps_trim.query("treat==0")[Y]

y_treat = sum(treated * weight_t) / len(ldw_cps_trim)
y_ntreat = sum(ntreated * weight_nt) / len(ldw_cps_trim)

y_treat - y_ntreat

-13423.217566097874

In [17]:
pd.Series(weight_t).describe()

count    185.000000
mean      14.399694
std       31.324742
min        1.920906
25%        2.178973
50%        3.647998
75%        8.549940
max      209.206654
Name: propensity_score, dtype: float64

In [18]:
pd.Series(weight_nt).describe()

count    15992.000000
mean         1.009458
std          0.049994
min          1.000529
25%          1.000529
50%          1.000811
75%          1.007456
max          2.085887
Name: propensity_score, dtype: float64

Apparently, we have weights that are too large in the treated group

### Stabilizing propensity weights

In [19]:
p_of_t = ldw_cps_trim["treat"].mean()

weight_t_stable = p_of_t / ldw_cps_trim.query("treat==1")["propensity_score"]
weight_nt_stable = p_of_t / (1 - ldw_cps_trim.query("treat==0")["propensity_score"])

treated = ldw_cps_trim.query("treat==1")[Y]
ntreated = ldw_cps_trim.query("treat==0")[Y]

nt = treated.shape[0]
nc = ntreated.shape[0]

y_treat = sum(treated * weight_t_stable) / nt
y_ntreat = sum(ntreated * weight_nt_stable) / nc

y_treat - y_ntreat

1142.896253187655